# Basic imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

# Loading datasets

In [2]:
cns_df=pd.read_csv("cns_molecules.csv", sep="\t")
non_cns_df=pd.read_csv("non_cns_molecules.csv", sep="\t")

In [3]:
cns_df_length=len(cns_df)
non_cns_df_length=len(non_cns_df)

print("cns rows: {}".format(cns_df_length))
print("non cns rows: {}".format(non_cns_df_length))

cns rows: 316
non cns rows: 624


# New column for both datasets (1= true, 0=false)

In [4]:
new_cns_column=[1 for i in range(cns_df_length)]
new_non_cns_columns=[0 for i in range(non_cns_df_length)]

In [5]:
cns_df["p_meninges"]=new_cns_column
non_cns_df["p_meninges"]=new_non_cns_columns

### Merged dataset

In [6]:
m_df=cns_df
m_df=m_df.append(non_cns_df)